## Eight Queens Puzzle

https://en.wikipedia.org/wiki/Eight_queens_puzzle

The N-Queens puzzle refers to a set of related questions that stem from the challenge of arranging eight queens on a standard 8x8 chess board in such a way that no two queeens are attacking one another.  The more general problem extends this to N queens on an NxN chess board.  Except for N=2 and N=3, this is know to have solutions for all positive N.

Some of the different forms of the challenge are:
1. Find all of the arrangements of queens that satisfy the condition.
2. Just count all of the arrangements of queens that satisfy the condition.
3. Find at least *one* arrangment of queens that satify the condition.

The second challenge (counting) has been [solved up to N=27](https://github.com/preusser/q27). (The number of solutions grows exponentially with N.)

The third challenge (finding at least one arrangement) can solved for much, much larger values of N.  (Solutions up to N=500,000 have been [reported.](http://citeseerx.ist.psu.edu/viewdoc/download;jsessionid=4DC9292839FE7B1AFABA1EDB8183242C?doi=10.1.1.57.4685&rep=rep1&type=pdf))

**Utility functions**

In [1]:
import timeit

def timings(fn, maxtime=10, runs=0):
    """Time the given function, fn(N), for increasing values of N, from N=6.
    This continues until the calculation time exceeds `maxtime` seconds.
    If a non-zero value for `runs` is given, it is evaluted that many times
    for each value of N.  If `runs` is zero, we try to pick a number of
    evaluations that totals to roughly 1 second.
    
    A list of (n, timing) tuples is returned.
    """
    result = []
    n, exec_time = 6, 0.001
    while exec_time < maxtime:
        nrun = runs or max(1, int(0.2/exec_time))  # assume time increases ~5x each round.
        exec_time = timeit.timeit("fn(n)", number=nrun, globals=locals()) / nrun
        result.append((n, exec_time))
        n += 1              
    return result

In [2]:
import numpy as np

def expfit(n, y):
    """Fit the (n, y) data to a simple exponential model y = exp(a*x + b)
    and print the results.  The factor exp(a) and offset b are displayed,
    along with the predictions for each point.
    
    A list of (n, timing, predicted timing) tuples is also returned.
    """
    cf = np.polyfit(n, np.log(y), 1)
    print("exp(a): {:3f}\nb: {}".format(np.exp(cf[0]), cf[1]))
    yp = np.exp(np.polyval(cf, n))
    print("\n".join(["{:2d}: {:6f}  pred: {:6f}".format(*val) for val in zip(n, y, yp)]))
    return list(zip(n, y, yp))

In [3]:
from bokeh.plotting import figure, output_notebook, show
from bokeh.palettes import Category20 
output_notebook()

def plot_timings(*named_timings, log=True):
    """Plot the given timings, each a tuple of the function name and
    a list of timing data (n, timing, fitted timing).
    """
    args = {'title': "Timings",
            'x_axis_label': 'N',
            'y_axis_label': 'time (sec)'}
    if log:
        args['y_axis_type'] = 'log'

    fig = figure(**args)
    
    # add a line renderer with legend and line thickness
    for i, timing in enumerate(named_timings):
        name, xyp = timing
        if len(xyp[0]) == 3:
            n, y, yp = zip(*xyp)
            fig.line(n, y, legend=name, line_width=2, color=Category20[20][2*i])
            fig.line(n, yp, legend=name + " (pred)", line_width=1, color=Category20[20][2*i+1])
        elif len(xyp[0]) == 2:
            n, y = zip(*xyp)
            fig.line(n, y, legend=name, line_width=2, color=Category20[20][2*i])
        else:
            raise ValueError("Can't decipher timing")

    # show the results
    show(fig)

Loading BokehJS ...

In [4]:
def print_solutions(solns):
    for i, b in enumerate(solns):
        print("{:2d}: {}".format(i+1, " ".join(map(str, b))))

### 1. Consider all possible arrangements of the N queens on distinct rows and columns.
* This is the most reasonable "brute-force" solution to the problem, because we know that, for every solution, the column numbers of the queens will be some permutation of the integers 1 to N.  We can generate those N! permutations directly.
* The main work here is to evaluate whether each board is a solution or not.

In [5]:
NQUEEN = 8

In [6]:
from itertools import permutations

def attack(b):
    """Return True if any two queens attack each other."""
    for i in range(len(b)-1):
        for j in range(i+1, len(b)):
            if abs(b[i] - b[j]) == j - i:
                return True
    return False
    
def solution1(nqueen):
    solns = []
    for board in permutations(list(range(nqueen))):
        if not attack(board):
            solns.append(tuple(board))
    return solns

solns = solution1(NQUEEN)
print("Found {} solutions".format(len(solns)))
# print_solutions(solns)

Found 92 solutions


In [7]:
time1 = timings(solution1)
n, y = zip(*time1)
pred1 = expfit(n, y)

exp(a): 8.963948
b: -19.59004929231573
 6: 0.001817  pred: 0.001611
 7: 0.013543  pred: 0.014443
 8: 0.114952  pred: 0.129463
 9: 1.103358  pred: 1.160496
10: 11.651505  pred: 10.402629


### 2. Find solutions using an exhaustive depth-first search.
* Doing a classic tree search for solutions should let us reduce the number of positions we need to look at, significantly, because it lets us backtrack as soon as we find that some queen placement prevents further queens from being placed.
* In principle, a breadth-first search wouldn't examine more positions than this depth-first search, but it would require storing many more intermediate states.  Since we want to find all solutions, anyway, a depth-first search makes more sense.

#### a. Reuse the *attack()* function from the brute-force search

In [8]:
def successors2(board, nqueen):
    """Return a generator yielding all new boards reachable from the
    input state by placing a queen somewhere on the next open row.
    """
    return [board + [j] for j in range(nqueen)
            if j not in board and not attack(board + [j])]

def solution2(nqueen):
    solns = []
    queue = successors2([], nqueen)
    while queue:
        b = queue.pop()
        if len(b) == nqueen:
            solns.append(b)
        else:
            queue.extend(successors2(b, nqueen))
    return solns

              
solns = solution2(NQUEEN)
print("Found {} solutions".format(len(solns)))

Found 92 solutions


In [9]:
time2 = timings(solution2)
n, y = zip(*time2)
pred2 = expfit(n, y)

exp(a): 5.277850
b: -16.55734601258002
 6: 0.001640  pred: 0.001393
 7: 0.007328  pred: 0.007353
 8: 0.034752  pred: 0.038805
 9: 0.180511  pred: 0.204809
10: 0.998932  pred: 1.080951
11: 5.792032  pred: 5.705098
12: 34.652195  pred: 30.110649


#### b. Only check attacks on newly added queens 
* When generating successor states, we really just need to check that each newly added queen isn't under attack from a previously placed queen.
* This is faster and scales better than the *attack()* function.

In [10]:
def adds_attack(b, col):
    """Return True if a queen added at some column on the next row is under attack.
    This should be all we need to check, since the input board b contains no attacks.
    """
    row = len(b)
    return any([abs(icol - col) == row - irow
                for irow, icol in enumerate(b)])

def successors2b(board, nqueen):
    return [board + [j] for j in range(nqueen)
            if j not in board and not adds_attack(board, j)]

def solution2b(nqueen):
    solns = []
    queue = successors2b([], nqueen)
    while queue:
        b = queue.pop()
        if len(b) == nqueen:
            solns.append(b)
        else:
            queue.extend(successors2b(b, nqueen))
    return solns
              
solns = solution2b(NQUEEN)
print("Found {} solutions".format(len(solns)))

Found 92 solutions


In [11]:
time2b = timings(solution2b)
n, y = zip(*time2b)
pred2b = expfit(n, y)

exp(a): 4.678310
b: -16.259707521682543
 6: 0.001080  pred: 0.000910
 7: 0.004214  pred: 0.004257
 8: 0.018134  pred: 0.019916
 9: 0.082581  pred: 0.093175
10: 0.390390  pred: 0.435902
11: 2.004094  pred: 2.039284
12: 11.437160  pred: 9.540401


### 3. Track search state using tuples of bit vectors.
* The state of a partially completed board is represented by three bit vectors that show the columns under attack by previously placed queens.
    - One vector shows the columns in which a queen has been placed,
    - One vector shows columns attacked diagonally from the right by a previously placed queen,
    - One vector shows columns attacked diagonally from the left.
* If we want to be able to show the final arrangement of queens, we also need to record the location of the queen in each row.

Ref: **Martin Richards** (Cambridge), [Backtracking Algorithms in MCPL
using Bit Patterns and Recursion](http://citeseerx.ist.psu.edu/viewdoc/download?doi=10.1.1.51.7113&rep=rep1&type=pdf)

In [12]:
from collections import namedtuple

def bits(i, nbits=NQUEEN):
    """Return a string containing the nbits-binary representation of integer i."""
    return bin((1 << nbits) | i)[3:]

Queens = namedtuple('Queens', ['col', 'rd', 'ld', 'loc'])
Queens.__repr__ = lambda q: "{}, {}, {} ({})".format(bits(q.col), bits(q.rd), bits(q.ld), ",".join(map(str, q.loc)))

def successors3(state, nqueen):
    col = nqueen - 1
    newq = 1 << col
    excl = state.col | state.ld | state.rd
    while newq:
        if not (newq & excl):
            yield Queens(state.col | newq,(state.rd | newq) >> 1, (state.ld | newq) << 1, state.loc + (col,))
        col -= 1
        newq = newq >> 1

In [13]:
# Verify that successors3() works the way we expect it to.
q0 = Queens(0, 0, 0, ())
print(q0)
print('- - - - - -')
for q in successors3(q0, NQUEEN):
    print(q)
print('===========')
q1 = list(successors3(q0, NQUEEN))[3]
print(q1)
print('- - - - - -')
for q in successors3(q1, NQUEEN):
    print(q)
print('===========')
q2 = list(successors3(q1, NQUEEN))[0]
print(q2)
print('- - - - - -')
for q in successors3(q2, NQUEEN):
    print(q)

00000000, 00000000, 00000000 ()
- - - - - -
10000000, 01000000, 00000000 (7)
01000000, 00100000, 10000000 (6)
00100000, 00010000, 01000000 (5)
00010000, 00001000, 00100000 (4)
00001000, 00000100, 00010000 (3)
00000100, 00000010, 00001000 (2)
00000010, 00000001, 00000100 (1)
00000001, 00000000, 00000010 (0)
00010000, 00001000, 00100000 (4)
- - - - - -
10010000, 01000100, 01000000 (4,7)
01010000, 00100100, 11000000 (4,6)
00010100, 00000110, 01001000 (4,2)
00010010, 00000101, 01000100 (4,1)
00010001, 00000100, 01000010 (4,0)
10010000, 01000100, 01000000 (4,7)
- - - - - -
10110000, 00110010, 111000000 (4,7,5)
10011000, 00100110, 110010000 (4,7,3)
10010010, 00100011, 110000100 (4,7,1)
10010001, 00100010, 110000010 (4,7,0)


#### a. Find all solutions

In [14]:
def solution3(nqueen):
    solns = []
    complete = (1 << nqueen) - 1
    queue = [q for q in successors3(Queens(0, 0, 0, ()), nqueen)]
    while queue:
        q = queue.pop()       
        if q.col == complete:
            solns.append(q.loc)
        else:
            for newq in successors3(q, nqueen):
                queue.append(newq)
    return solns
              
solns = solution3(NQUEEN)
print("Found {} solutions".format(len(solns)))

Found 92 solutions


In [15]:
time3 = timings(solution3)
n, y = zip(*time3)
pred3 = expfit(n, y)

exp(a): 4.489285
b: -16.609297942767267
 6: 0.000667  pred: 0.000501
 7: 0.002326  pred: 0.002249
 8: 0.008895  pred: 0.010095
 9: 0.036683  pred: 0.045318
10: 0.168084  pred: 0.203444
11: 0.828152  pred: 0.913318
12: 4.261791  pred: 4.100144
13: 24.058910  pred: 18.406713


#### b. Just count the number of solutions
* Keeping track of which queens are in which columns isn't necessary if you just want to count the number of solutions.  So, we can simplify things a little by not keeping track of that.
* The speedup from this is much less than I expected. (~8%)

In [16]:
Queens2 = namedtuple('Queens', ['col', 'rd', 'ld'])
Queens2.__repr__ = lambda q: "{}, {}, {}".format(bits(q.col), bits(q.rd), bits(q.ld))

def successors3b(state, nqueen):
    col = nqueen - 1
    newq = 1 << col
    excl = state.col | state.ld | state.rd
    while newq:
        if not (newq & excl):
            yield Queens2(state.col | newq, (state.rd | newq) >> 1, (state.ld | newq) << 1)
        col -= 1
        newq = newq >> 1
        
def solution3b(nqueen):
    count = 0
    complete = (1 << nqueen) - 1
    queue = [q for q in successors3b(Queens2(0, 0, 0), nqueen)]
    while queue:
        q = queue.pop()       
        if q.col == complete:
            count += 1
        else:
            for newq in successors3b(q, nqueen):
                queue.append(newq)
    return count
              
nsolns = solution3b(NQUEEN)
print("Found {} solutions".format(nsolns))

Found 92 solutions


In [17]:
time3b = timings(solution3b)
n, y = zip(*time3b)
pred3b = expfit(n, y)

exp(a): 4.492821
b: -16.695693119441998
 6: 0.000627  pred: 0.000462
 7: 0.002114  pred: 0.002074
 8: 0.008133  pred: 0.009318
 9: 0.034220  pred: 0.041862
10: 0.155686  pred: 0.188080
11: 0.740646  pred: 0.845011
12: 4.024793  pred: 3.796483
13: 22.447378  pred: 17.056917


#### c. Replace namedtuples with simple tuples
* I was just curious how much overhead there was in using namedtuples.  It turns out it's significant. (~2x speedup)

In [18]:
def successors3c(col, rd, ld, nqueen):
    newq = 1 << (nqueen - 1)
    excl = col | rd | ld
    while newq:
        if not (newq & excl):
            yield (col | newq, (rd | newq) >> 1, (ld | newq) << 1)
        newq = newq >> 1
        
def solution3c(nqueen):
    count = 0
    done = (1 << nqueen) - 1
    queue = list(successors3c(0, 0, 0, nqueen))
    while queue:
        col, rd, ld = queue.pop()       
        if col == done:
            count += 1
        else:
            queue.extend(successors3c(col, rd, ld, nqueen))
    return count
              
nsolns = solution3c(NQUEEN)
print("Found {} solutions".format(nsolns))

Found 92 solutions


In [19]:
time3c = timings(solution3c)
n, y = zip(*time3c)
pred3c = expfit(n, y)

exp(a): 4.480427
b: -17.16838004717312
 6: 0.000388  pred: 0.000283
 7: 0.001283  pred: 0.001268
 8: 0.004987  pred: 0.005681
 9: 0.020873  pred: 0.025453
10: 0.093134  pred: 0.114041
11: 0.448964  pred: 0.510953
12: 2.376548  pred: 2.289288
13: 13.798115  pred: 10.256988


#### d. Optimize the iteration over bits in the bitvectors
* Instead of checking bit by bit for available positions for the queen on each row, we can directly generate bit-masks for each available position.
* This significantly reduces the number of positions we need to iterate over and gives a 2x speedup.

In [20]:
def solution3d(nqueen):
    count = 0
    done = (1 << nqueen) - 1
    
    def successors3d(col, rd, ld):
        nonlocal done
        avail = ~(col | rd | ld) & done
        while avail:
            newq = avail & -avail
            yield (col | newq, (rd | newq) >> 1, (ld | newq) << 1)
            avail -= newq

    queue = list(successors3d(0, 0, 0))
    while queue:
        col, rd, ld = queue.pop()       
        if col == done:
            count += 1
        else:
            queue.extend(successors3d(col, rd, ld))
    return count
              
nsolns = solution3d(NQUEEN)
print("Found {} solutions".format(nsolns))

Found 92 solutions


In [21]:
time3d = timings(solution3d)
n, y = zip(*time3d)
pred3d = expfit(n, y)

exp(a): 4.435498
b: -17.51507951911013
 6: 0.000285  pred: 0.000188
 7: 0.000884  pred: 0.000835
 8: 0.003263  pred: 0.003705
 9: 0.012427  pred: 0.016435
10: 0.060000  pred: 0.072899
11: 0.253134  pred: 0.323343
12: 1.368514  pred: 1.434187
13: 6.808153  pred: 6.361334
14: 40.267641  pred: 28.215683


### 4. Replace iterative search with a recursive search
* This doesn't reduce the number of positions that are examined, but it avoids having to manage a queue.  This both simplifies the logic a little, and reduces memory use.

In [31]:
def solution4(nqueen):
    done = (1 << nqueen) - 1
    count = 0

    def count4(col, rd, ld):
        nonlocal count
        if col == done:
            count += 1
            return
        avail = ~(col | rd | ld) & done
        while avail:
            newq = avail & -avail
            count4(col | newq, (rd | newq) >> 1, (ld | newq) << 1)
            avail -= newq
        
    count4(0, 0, 0)
    return count

              
nsolns = solution4(NQUEEN)
print("Found {} solutions".format(nsolns))

Found 92 solutions


In [32]:
time4 = timings(solution4, maxtime=5)
n, y = zip(*time4)
pred4 = expfit(n, y)

exp(a): 4.504990
b: -18.3488073577283
 6: 0.000127  pred: 0.000090
 7: 0.000450  pred: 0.000405
 8: 0.001658  pred: 0.001823
 9: 0.006770  pred: 0.008212
10: 0.027383  pred: 0.036996
11: 0.131270  pred: 0.166666
12: 0.677486  pred: 0.750827
13: 3.795423  pred: 3.382468
14: 21.811975  pred: 15.237985


#### Alex's solution from Stackoverflow
* Back in April 2017, Alex posted this solution, [N-Queens, bitwise approach](https://codereview.stackexchange.com/questions/159946/n-queens-bitwise-approach)
* My *solution4()*, above, is basically identical to this, just with different notation.

In [26]:
def solutionAlex(n):
    all_ones = 2 ** n - 1
    count = 0

    def helper(ld, column, rd):
        nonlocal count
        if column == all_ones:
            count += 1
            return
        possible_slots = ~(ld | column | rd) & all_ones
        while possible_slots:
            current_bit = possible_slots & -possible_slots
            possible_slots -= current_bit
            helper((ld | current_bit) >> 1,
                   column | current_bit,
                   (rd | current_bit) << 1)

    helper(0, 0, 0)
    return count

nsolns = solutionAlex(NQUEEN)
print("Found {} solutions".format(nsolns))

Found 92 solutions


In [27]:
timeAlex = timings(solutionAlex, maxtime=5)
n, y = zip(*timeAlex)
predAlex = expfit(n, y)

exp(a): 4.437167
b: -18.139253071578548
 6: 0.000174  pred: 0.000101
 7: 0.000463  pred: 0.000449
 8: 0.001628  pred: 0.001991
 9: 0.006785  pred: 0.008834
10: 0.028486  pred: 0.039199
11: 0.136013  pred: 0.173935
12: 0.716350  pred: 0.771777
13: 3.967161  pred: 3.424505
14: 22.352117  pred: 15.195103


### Plot timings for the different solutions

#### Python solutions

In [46]:
plot_timings(('soln1', time1), ('soln2', time2), ('soln2b', time2b), ('soln3c', time3c), ('soln3d', time3d), ('soln4', time4))

#### Compiled languages
* I implemented the fastest bitwise solution in both Go and C, as well.
* These languages both compile directly to machine code, and are generally much faster than Python.
* Here, the C code runs ~10% faster than the Go code.

In [39]:
timeGo = [(18, 610),(17, 84.9),(16, 12.2),(15, 1.83),(14, 0.291),(13, 0.061),(12, 0.021)]
n, y = zip(*timeGo)
predGo = expfit(n, y) 

exp(a): 5.763376
b: -25.406259315050363
18: 610.000000  pred: 455.393261
17: 84.900000  pred: 79.015022
16: 12.200000  pred: 13.709851
15: 1.830000  pred: 2.378788
14: 0.291000  pred: 0.412742
13: 0.061000  pred: 0.071615
12: 0.021000  pred: 0.012426


In [42]:
timeC = [(18, 557), (17, 75.2),(16, 10.9),(15, 1.64),(14, 0.267),(13, 0.102),(12, 0.02)]
n, y = zip(*timeC)
predC = expfit(n, y) 

exp(a): 5.477644
b: -24.651431051674084
18: 557.000000  pred: 387.889280
17: 75.200000  pred: 70.813162
16: 10.900000  pred: 12.927668
15: 1.640000  pred: 2.360078
14: 0.267000  pred: 0.430856
13: 0.102000  pred: 0.078657
12: 0.020000  pred: 0.014360


In [47]:
plot_timings(('soln1', time1), ('soln2', time2), ('soln2b', time2b), ('soln3c', time3c), ('soln3d', time3d), ('soln4', time4), ('solnGo', timeGo), ('solnC', timeC))